In [9]:
import numpy as np
import torch
import os, sys, time

In [10]:
def forloopdists(X, T):
    D = np.zeros((X.shape[0], T.shape[0]))

    for i in range(X.shape[0]):
        for j in range(T.shape[0]):
            D[i, j] = np.linalg.norm(X[i, :] - T[j, :])**2
    return D

In [11]:
def numpydists(X, T):
    X = np.expand_dims(X, axis=1)
    T = np.expand_dims(T, axis=0)
    
    X2 = np.einsum("ijk, ijk -> ij", X, X)
    T2 = np.einsum("ijk, ijk -> ij", T, T)
    XT = np.einsum("ijk, ijk -> ij", X, T)
    
    return X2 + T2 - 2 * XT

In [12]:
def pytorchdists(X, T):
    X = torch.tensor(X, device = "cuda")
    T = torch.tensor(T, device = "cuda")
    XT = torch.mm(X, T.T)
    
    X = torch.unsqueeze(X, dim=1)
    T = torch.unsqueeze(T, dim=0)
    
    X2 = torch.einsum("ijk, ijk -> ij", X, X)
    T2 = torch.einsum("ijk, ijk -> ij", T, T)
    
    return X2 + T2 - 2 * XT

In [13]:
X = np.random.normal(size=(500, 30)) #5000 instead of 250k for forloopdists
T = np.random.normal(size=(50, 30))

D3 = numpydists(X, T)
D = pytorchdists(X, T).to("cpu")

In [14]:
print(D - D3)

tensor([[ 7.1054e-15, -7.1054e-15,  7.1054e-15,  ...,  0.0000e+00,
          1.4211e-14,  0.0000e+00],
        [ 0.0000e+00, -7.1054e-15, -7.1054e-15,  ...,  7.1054e-15,
          7.1054e-15, -1.4211e-14],
        [ 1.4211e-14,  0.0000e+00, -7.1054e-15,  ...,  0.0000e+00,
         -7.1054e-15, -3.5527e-15],
        ...,
        [-7.1054e-15, -2.1316e-14, -1.4211e-14,  ..., -1.4211e-14,
          0.0000e+00, -1.4211e-14],
        [ 1.4211e-14,  0.0000e+00, -7.1054e-15,  ...,  0.0000e+00,
         -7.1054e-15,  0.0000e+00],
        [ 7.1054e-15,  0.0000e+00, -7.1054e-15,  ..., -7.1054e-15,
          0.0000e+00, -7.1054e-15]], dtype=torch.float64)


## The Test

In [15]:
def timing():
    X = np.random.normal(size=(5000, 300)) #5000 instead of 250k for forloopdists
    T = np.random.normal(size=(500, 300))

    #if X.shape[0] * T.shape[0] * X.shape[1] < 1E9:
    #    since = time.time()
    #    dists0 = forloopdists(X, T)
    #    time_elapsed = float(time.time()) - float(since)
    #    print('For complete in {:.3f}s'.format( time_elapsed ))

    since = time.time()
    dists1 = pytorchdists(X, T)
    time_elapsed = float(time.time()) - float(since)
    print('Torch complete in {:.3f}s'.format( time_elapsed ))

    since = time.time()
    dists2 = numpydists(X, T)
    time_elapsed = float(time.time()) - float(since)
    print('Numpy complete in {:.3f}s'.format( time_elapsed ))

In [16]:
timing()

Torch complete in 0.002s
Numpy complete in 0.342s
